
# Principal Componenet Analysis (PCA)
The PCA algorithm is a dimensionality reduction algorithm which works really well for datasets which have correlated columns. It combines the features of X in linear combination such that the new components capture the most information of the data. 

The PCA model is implemented in the cuML library and can accept the following parameters: 
1. `svd_solver`: selects the type of algorithm used: Jacobi or full (default = full)
2. `n_components`: the number of top K vectors to be present in the output (default = 1)
3. `random_state`: select a random state if the results should be reproducible across multiple runs (default = None)
4. `copy`: if 'True' then it copies the data and removes mean from it else the data will be overwritten with its mean centered version (default = True)
5. `whiten`: if True, de-correlates the components (default = False)
6. `tol`: if the svd_solver = 'Jacobi' then this variable is used to set the tolerance (default = 1e-7)
7. `iterated_power`: if the svd_solver = 'Jacobi' then this variable decides the number of iterations (default = 15)

The cuml implementation of the PCA model has the following functions that one can run:
1. `Fit`: it fits the model with the dataset
2. `Fit_transform`: fits the PCA model with the dataset and performs dimensionality reduction on it
3. `Inverse_transform`: returns the original dataset when the transformed dataset is passed as the input
4. `Transform`: performs dimensionality reduction on the dataset
5. `Get_params`: returns the value of the parameters of the PCA model
6. `Set_params`: allows the user to set the value of the parameters of the PCA model

The model accepts only numpy arrays or cudf dataframes as the input. 
  - To convert your dataset to cudf format please read the cudf [documentation](https://rapidsai.github.io/projects/cudf/en/latest/). 
  - For additional information on the PCA model please refer to the [documentation](https://rapidsai.github.io/projects/cuml/en/latest/index.html)

In [1]:
!wget -nc https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
!bash rapids-colab.sh

import sys, os

sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

--2019-07-27 09:48:46--  https://github.com/rapidsai/notebooks-extended/raw/master/utils/rapids-colab.sh
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/utils/rapids-colab.sh [following]
--2019-07-27 09:48:47--  https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/utils/rapids-colab.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1548 (1.5K) [text/plain]
Saving to: ‘rapids-colab.sh’

rapids-colab.sh     100%[===================>]   1.51K  --.-KB/s    in 0s      

2019-07-27 09:48:47 (225 MB/s) - ‘rapids-colab.sh’ saved [1548/1548]

--2019-07-27 09

In [0]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA as skPCA
from cuml import PCA as cumlPCA
import cudf
import os

In [3]:
#https://github.com/rapidsai/notebooks/blob/branch-0.8/cuml/data/mortgage.npy.gz?raw=true
!mkdir -p data/ && wget -O data/mortgage.npy.gz https://github.com/rapidsai/notebooks/blob/branch-0.8/cuml/data/mortgage.npy.gz?raw=true

--2019-07-27 09:54:22--  https://github.com/rapidsai/notebooks/blob/branch-0.8/cuml/data/mortgage.npy.gz?raw=true
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/rapidsai/notebooks/raw/branch-0.8/cuml/data/mortgage.npy.gz [following]
--2019-07-27 09:54:22--  https://github.com/rapidsai/notebooks/raw/branch-0.8/cuml/data/mortgage.npy.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks/branch-0.8/cuml/data/mortgage.npy.gz [following]
--2019-07-27 09:54:23--  https://raw.githubusercontent.com/rapidsai/notebooks/branch-0.8/cuml/data/mortgage.npy.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|

# Helper Functions

In [0]:
# check if the mortgage dataset is present and then extract the data from it, else do not run 
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        # throws FileNotFoundError error if mortgage dataset is not present
        raise FileNotFoundError('Please download the required dataset or check the path')
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    return df

In [0]:
# this function checks if the results obtained from two different methods (sklearn and cuml) are the equal
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

# the function converts a variable from ndarray or dataframe format to numpy array
def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

# Run tests

In [6]:
%%time
# nrows = number of samples
# ncols = number of features of each sample

nrows = 2**15
nrows = int(nrows * 1.5)
ncols = 400

X = load_data(nrows,ncols)
#print('data',X.shape) <- this line currently throws an error, tuple takes no attribute shape
#print(X)

use mortgage data
CPU times: user 5.26 s, sys: 976 ms, total: 6.24 s
Wall time: 6.27 s


In [0]:
# set parameters for the PCA model
n_components = 10
whiten = False
random_state = 42
svd_solver="full"

In [8]:
%%time
# convert the pandas dataframe to cudf dataframe
X = cudf.DataFrame.from_pandas(X)

CPU times: user 1.31 s, sys: 498 ms, total: 1.8 s
Wall time: 1.95 s


In [9]:
%%time
# use the sklearn PCA on the dataset
pca_sk = skPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
# creates adwdwn embedding
result_sk = pca_sk.fit_transform(to_nparray(X))

CPU times: user 3.07 s, sys: 186 ms, total: 3.25 s
Wall time: 2.03 s


In [10]:
%%time
# use the cuml PCA model on the dataset
pca_cuml = cumlPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
# obtain the embedding of the model
result_cuml = pca_cuml.fit_transform(X)

CPU times: user 1.21 s, sys: 107 ms, total: 1.32 s
Wall time: 1.28 s


In [11]:
# calculate the attributes of the two models and compare them
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(pca_sk,attr),getattr(pca_cuml,attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

compare pca: cuml vs sklearn          singular_values_ equal
compare pca: cuml vs sklearn               components_ equal
compare pca: cuml vs sklearn       explained_variance_ equal
compare pca: cuml vs sklearn explained_variance_ratio_ equal


In [12]:
# compare the results of the two models
passed = array_equal(result_sk,result_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

compare pca: cuml vs sklearn transformed results equal
